<a href="https://colab.research.google.com/github/scadusseau/Planification/blob/master/ValueIteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import pymdptoolbox module

In [ ]:
! pip install pymdptoolbox

  Created wheel for pymdptoolbox: filename=pymdptoolbox-4.0b3-cp36-none-any.whl size=25659 sha256=4625791ac7f08a8cc0eaeae06b8eca9ae14f16f35ce951667f9cd7d30b71bc8b
  Stored in directory: /root/.cache/pip/wheels/87/a9/a8/40c4e252c02e590737265742425cdd0365fafcc162441a9527
Successfully built pymdptoolbox


#Few code lines, to check if the import works

In [ ]:
import mdptoolbox.example
P, R = mdptoolbox.example.forest()
vi = mdptoolbox.mdp.ValueIteration(P, R, 0.9)
vi.run()
vi.policy # result is (0, 0, 0)
P
#type(P)

array([[[0.1, 0.9, 0. ],
        [0.1, 0. , 0.9],
        [0.1, 0. , 0.9]],

       [[1. , 0. , 0. ],
        [1. , 0. , 0. ],
        [1. , 0. , 0. ]]])

#Example of Value Iteration Algorithm with Maze environment

In [ ]:
"""
Code adapted from C. Chanel's lecture


(2, 2, 2, 0, 0, 3, 0, 0, 3, 3, 1) NSEW
(0, 0, 0, 0, 2, 3, 0, 2, 3, 3, 1) ESNW
(-0.21123300369199005, -0.15325427305044126, -0.09754731733007559, 0.0, -0.25992232445174374, -0.40295075982861484, -99.98970739403872, -0.3105972352765278, -0.3531529614940261, -0.39411776013168864, -0.6030426704300679)
% --------------
% |s0 s1 s2 s3+ |
% |s4  # s5 S6- |
% |s7 s8 s9 s10 |
% --------------



               s0   s1  s2  s3  s4  s5  s6  s7  s8  s9 s10 
         s0   [0.9 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
         s1    0.1 0.8 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
         s2    0.0 0.1 0.8 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
         s3    0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
         s4    0.8 0.0 0.0 0.0 0.2 0.0 0.0 0.0 0.0 0.0 0.0;
         s5    0.0 0.0 0.8 0.0 0.0 0.1 0.1 0.0 0.0 0.0 0.0;
         s6    0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0;
         s7    0.0 0.0 0.0 0.0 0.8 0.0 0.0 0.1 0.1 0.0 0.0;
         s8    0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.1 0.8 0.1 0.0;
         s9    0.0 0.0 0.0 0.0 0.0 0.8 0.0 0.0 0.1 0.0 0.1;
        s10    0.0 0.0 0.0 0.0 0.0 0.0 0.8 0.0 0.0 0.1 0.1];
"""

import numpy as np
#addpath('/home/cchanel/Documents/Cours/Planification CNAM parcours drone/MDPtoolbox/MDPtoolbox/')

# MDP MAZE EXAMPLE
#####################
# --------------
# |s0 s1 s2 s3+ |
# |s4  # s5 S6- |
# |s7 s8 s9 s10 |
# --------------
# The + indicates a reward of 1.0, the - a penalty of -1.0.
# The # in the middle of the maze is an obstruction.
# Rewards and penalties are associated with states, not actions.
# The default reward/penalty is -0.04.
# There is no discounting, but a there is an absorbing state that
# + and - transition to automatically.  The absorbing state cannot be exited.
# 
# 
# The actions, NSEW, have the expected result 80% of the time, and
# transition in a direction perpendicular to the intended on with a 10%
# probability for each direction.  Movement into a wall returns the agent
# to its original state.
# 
# good = +1 reward, bad = -1 penalty,

# north
"""
P(:,:,1) = [ 0.9 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
             0.1 0.8 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
             0.0 0.1 0.8 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
             0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
             0.8 0.0 0.0 0.0 0.2 0.0 0.0 0.0 0.0 0.0 0.0;
             0.0 0.0 0.8 0.0 0.0 0.1 0.1 0.0 0.0 0.0 0.0;
             0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0;
             0.0 0.0 0.0 0.0 0.8 0.0 0.0 0.1 0.1 0.0 0.0;
             0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.1 0.8 0.1 0.0;
             0.0 0.0 0.0 0.0 0.0 0.8 0.0 0.0 0.1 0.0 0.1;
             0.0 0.0 0.0 0.0 0.0 0.0 0.8 0.0 0.0 0.1 0.1];
"""
N = [[ 0.9, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.1, 0.8, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.1, 0.8, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.8, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.0, 0.8, 0.0, 0.0, 0.1, 0.1, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.1, 0.1, 0.0, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.8, 0.1, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.1, 0.0, 0.1],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.1, 0.1]]

# east
"""
P(:,:,2) = [0.1 0.8 0.0 0.0 0.1 0.0 0.0 0.0 0.0 0.0 0.0;
            0.0 0.2 0.8 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
            0.0 0.0 0.1 0.8 0.0 0.1 0.0 0.0 0.0 0.0 0.0;
            0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
            0.1 0.0 0.0 0.0 0.8 0.0 0.0 0.1 0.0 0.0 0.0;
            0.0 0.0 0.1 0.0 0.0 0.0 0.8 0.0 0.0 0.1 0.0;
            0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0;
            0.0 0.0 0.0 0.0 0.1 0.0 0.0 0.1 0.8 0.0 0.0;
            0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.2 0.8 0.0;
            0.0 0.0 0.0 0.0 0.0 0.1 0.0 0.0 0.1 0.0 0.8;
            0.0 0.0 0.0 0.0 0.0 0.0 0.1 0.0 0.0 0.0 0.9];
"""
E = [[ 0.1, 0.8, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.2, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.0, 0.1, 0.8, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.1, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0],
     [ 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.1, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.1, 0.8, 0.0, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.8, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.1, 0.0, 0.8],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.9]]

# south    
"""
P(:,:,3) = [ 0.1 0.1 0.0 0.0 0.8 0.0 0.0 0.0 0.0 0.0 0.0;
             0.1 0.8 0.1 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
             0.0 0.1 0.0 0.1 0.0 0.8 0.0 0.0 0.0 0.0 0.0;
             0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
             0.0 0.0 0.0 0.0 0.2 0.0 0.0 0.8 0.0 0.0 0.0;
             0.0 0.0 0.0 0.0 0.0 0.1 0.1 0.0 0.0 0.8 0.0;
             0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0;
             0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.9 0.1 0.0 0.0;
             0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.1 0.8 0.1 0.0;
             0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.1 0.8 0.1;
             0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.1 0.9];
"""
S = [[ 0.1, 0.1, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.1, 0.8, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.1, 0.0, 0.1, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.1, 0.0, 0.0, 0.8, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9, 0.1, 0.0, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.8, 0.1, 0.0],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.8, 0.1],
     [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.9]]
# west
"""
P(:,:,4) = [0.9 0.0 0.0 0.0 0.1 0.0 0.0 0.0 0.0 0.0 0.0;
            0.8 0.2 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
            0.0 0.8 0.1 0.0 0.0 0.1 0.0 0.0 0.0 0.0 0.0;
            0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;
            0.1 0.0 0.0 0.0 0.8 0.0 0.0 0.1 0.0 0.0 0.0;
            0.0 0.0 0.1 0.0 0.0 0.8 0.0 0.0 0.0 0.1 0.0;
            0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0;
            0.0 0.0 0.0 0.0 0.1 0.0 0.0 0.9 0.0 0.0 0.0;
            0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.8 0.2 0.0 0.0;
            0.0 0.0 0.0 0.0 0.0 0.1 0.0 0.0 0.8 0.1 0.0;
            0.0 0.0 0.0 0.0 0.0 0.0 0.1 0.0 0.0 0.8 0.1];
"""     
W = [[ 0.9, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
      [ 0.8, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
      [ 0.0, 0.8, 0.1, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
      [ 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
      [ 0.1, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0],
      [ 0.0, 0.0, 0.1, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.1, 0.0],
      [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
      [ 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.9, 0.0, 0.0, 0.0],
      [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.2, 0.0, 0.0],
      [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.8, 0.1, 0.0],
      [ 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.8, 0.1]]

P = np.array([N,S,E,W])

# Costs/Rewards -> columns are related to actions
R = np.array([[-0.05, -0.05, -0.05, -0.05],
              [-0.05, -0.05, -0.05, -0.05],
              [-0.05, -0.05, -0.05, -0.05],
              [ 0.00,  0.00,  0.00,  0.00],
              [-0.05, -0.05, -0.05, -0.05],
              [-0.05, -0.05, -0.05, -0.05],
              [-5.00, -5.00, -5.00, -5.00],
              [-0.05, -0.05, -0.05, -0.05],
              [-0.05, -0.05, -0.05, -0.05],
              [-0.05, -0.05, -0.05, -0.05],
              [-0.05, -0.05, -0.05, -0.05]])


P

"""
          North   East   West  South
Costs = [[-0.05, -0.05, -0.05, -0.05],    s0
         [-0.05, -0.05, -0.05, -0.05],    s1
         [-0.05, -0.05, -0.05, -0.05],    s2
         [ 0.00,  0.00,  0.00,  0.00],    s3 : Goal
         [-0.05, -0.05, -0.05, -0.05],    s4
         [-0.05, -0.05, -0.05, -0.05],    s5
         [-5.00, -5.00, -5.00, -5.00],    s6 : Dead-End
         [-0.05, -0.05, -0.05, -0.05],    s7
         [-0.05, -0.05, -0.05, -0.05],    s8
         [-0.05, -0.05, -0.05, -0.05],    s9
         [-0.05, -0.05, -0.05, -0.05]]    s10
"""
  

array([[[0.9, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0.1, 0.8, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0.1, 0.8, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0.8, 0. , 0. , 0. , 0.2, 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0.8, 0. , 0. , 0.1, 0.1, 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0. , 0.8, 0. , 0. , 0.1, 0.1, 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.1, 0.8, 0.1, 0. ],
        [0. , 0. , 0. , 0. , 0. , 0.8, 0. , 0. , 0.1, 0. , 0.1],
        [0. , 0. , 0. , 0. , 0. , 0. , 0.8, 0. , 0. , 0.1, 0.1]],

       [[0.1, 0.1, 0. , 0. , 0.8, 0. , 0. , 0. , 0. , 0. , 0. ],
        [0.1, 0.8, 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0.1, 0. , 0.1, 0. , 0.8, 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
        [0. , 0. , 0. ,

#Once Transitions and Rewards are defined, we can apply mdptoolbox algorithms

In [ ]:
# Computing policy using value iteration
# function [policy, iter, cpu_time] = mdp_value_iteration(P, R, discount, epsilon, max_iter, V0)
# initialization of values is V0=zeros(11,1);
# lauching policy computation

print('Applying Value Iteration')

#[policy, V, iter, cpu_time] = mdp_value_iteration(P,R, 0.95, 0.01)
V = mdptoolbox.mdp.ValueIteration(P,R, 0.95,0.01, 10000)
V.setVerbose()
V.run()
print(V)

print("##############################") 
# using policy interation
# function [V, policy, iter, cpu_time] = mdp_policy_iteration(P, R, discount, policy0, max_iter, eval_type)

print('Applying Policy Iteration')
#[V, policy, iter, cpu_time] = mdp_policy_iteration(P, R, 0.95)
D = mdptoolbox.mdp.PolicyIteration(P,R, 0.95)
D.setVerbose()
D.run()
D.policy

Applying Value Iteration
  Iteration		V-variation
    1		  5.0
    2		  4.75
    3		  4.512499999999999
    4		  4.286874999999998
    5		  4.072531250000001
    6		  3.868904687499999
    7		  3.6754594531250007
    8		  3.491686480468747
    9		  3.3171021564453156
    10		  3.1512470486230484
    11		  2.9936846961918917
    12		  2.8440004613823007
    13		  2.7018004383131853
    14		  2.5667104163975267
    15		  2.4383748955776454
    16		  2.316456150798764
    17		  2.200633343258829
    18		  2.0906016760958863
    19		  1.9860715922910899
    20		  1.8867680126765407
    21		  1.7924296120427101
    22		  1.7028081314405625
    23		  1.6176677248685394
    24		  1.536784338625111
    25		  1.4599451216938633
    26		  1.386947865609173
    27		  1.3176004723287065
    28		  1.2517204487122768
    29		  1.1891344262766665
    30		  1.1296777049628304
    31		  1.0731938197146889
    32		  1.0195341287289494
    33		  0.9685574222925055
    34		  0.9201295511778795
    35		  0

(2, 2, 2, 0, 0, 3, 0, 0, 3, 3, 1)

#Display the results

In [ ]:
print(V.policy)
print("\n\n##########################\n\n")
print(V.V)

(2, 2, 2, 0, 0, 3, 0, 0, 3, 3, 1)


##########################


(-0.21123300369199005, -0.15325427305044126, -0.09754731733007559, 0.0, -0.25992232445174374, -0.40295075982861484, -99.98970739403872, -0.3105972352765278, -0.3531529614940261, -0.39411776013168864, -0.6030426704300679)


#Now let's work on a basic SSP environment: Triangle Tire-World Problem

In [ ]:
import numpy as np
"""
TireWorld Problem:
A car has to reach the goal state, and can chose different roads. Some roads can lead to dead-ends -the car cannot advance anymore-, some road are safe, but increase the distance
to the goal. The goal of this exercise is to find balance between safety and speed.
The probability to reach a dead-end from an intermediate state is 0.5 when a dead end is directly linked to this state, for any action chosen. Else, the probability to reach a
non dead end state from another non dead state (that is not linked to a dead end state) is 1.0 (The probability to do an action and reach the expected state is 1.0).
The cost of the road between two states is 1.0

Actions: North, East, West (NEW)
19 states: 13 intermediate states, 4 dead ends {14, 15, 16, 17}, 1 initial state {S}, 1 Goal state {G}

North:  <--- 
East: \
West: /

             13
            /   \
           /     \
          /  17   \
         12   ^   11
        /   \ | /   \
       /     \|/     \
      10 <--- 9  <--- 8
     /  \   /   \   /  \
    /    \ /     \ /    \
   7      6       5      4
  /  \   /  \   /  \   /  \
 /    \ /    \ /    \ /    \
G <--- 3 <--- 2 <--- 1 <--- S(0)
       |      |      |
       |      |      |
      16     15      14

"""

"""
North:
     S     1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16   17   G
S   [0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
1   [0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.],
2   [0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.],
3   [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0., 0.5],
4   [0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
5   [0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
6   [0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
7   [0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
8   [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
9   [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.],
10  [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
11  [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
12  [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.],
13  [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.],
14  [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.],
15  [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.],
16  [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
17  [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.],
G   [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0],
"""

N = [[0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.],
     [0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0., 0.5],
     [0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0]]


# East
#     S     1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16   17   G
E = [[0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0., 0.5,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0]]


# West
#     S     1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16   17   G
W = [[1.0, 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0],
     [0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 0.5,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0,  0.],
     [0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 1.0]]



P = np.array([N,E,W])

# Costs/Rewards -> columns are related to actions
# Let's first define rewards so that the agent recieves a strong negative reward when it reaches a dead end state
R = np.array([[-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-10.00, -10.00, -10.00],
              [-10.00, -10.00, -10.00],
              [-10.00, -10.00, -10.00],
              [-10.00, -10.00, -10.00],
              [0.00,  0.00,  0.00]])

# Costs/Rewards -> columns are related to actions
#Let's see what happens if we don't accentuate the cost of a dead-end state.
C = np.array([[-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [0.00,  0.00,  0.00]])

#Now let's see what happens when we define a strong positive reward for the goal state
G = np.array([[-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-10.00, -10.00, -10.00],
              [-10.00, -10.00, -10.00],
              [-10.00, -10.00, -10.00],
              [-10.00, -10.00, -10.00],
              [10.00,  10.00,  10.00]])


#Now let's keep the strong positive rewards, but remove the strong negative rewards
L = np.array([[-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [-1.00, -1.00, -1.00],
              [10.00,  10.00,  10.00]])



#Let's run Value Iteration on this environment

In [ ]:
import mdptoolbox
# Computing policy using value iteration
# function [policy, iter, cpu_time] = mdp_value_iteration(P, R, discount, epsilon, max_iter, V0)
# initialization of values is V0=zeros(11,1);
# lauching policy computation
print('Applying Value Iteration')
#[policy, V, iter, cpu_time] = mdp_value_iteration(P,R, 0.95, 0.01)
V = mdptoolbox.mdp.ValueIteration(P,L, 0.95,0.01, 10000)
V.setVerbose()
V.run()
print(V)
print("##############################") 
# using policy interation
# function [V, policy, iter, cpu_time] = mdp_policy_iteration(P, R, discount, policy0, max_iter, eval_type)
print('Applying Policy Iteration')
#[V, policy, iter, cpu_time] = mdp_policy_iteration(P, R, 0.95)
D = mdptoolbox.mdp.PolicyIteration(P,L, 0.95)
D.setVerbose()
D.run()
D.policy

Applying Value Iteration
  Iteration		V-variation
    1		  11.0
    2		  10.45
    3		  9.927499999999998
    4		  9.431125
    5		  8.95956875
    6		  8.511590312500001
    7		  8.086010796875001
    8		  7.681710257031251
    9		  7.297624744179694
    10		  6.932743506970713
    11		  6.586106331622171
    12		  6.256801015041063
    13		  5.9439609642890066
    14		  5.646762916074559
    15		  5.364424770270835
    16		  5.096203531757295
    17		  4.841393355169432
    18		  4.599323687410964
    19		  4.369357503040412
    20		  4.150889627888402
    21		  3.943345146493977
    22		  3.7461778891692816
    23		  3.5588689947108154
    24		  3.3809255449752875
    25		  3.211879267726527
    26		  3.051285304340208
    27		  2.8987210391231972
    28		  2.7537849871670375
    29		  2.6160957378086653
    30		  2.485290950918227
    31		  2.361026403372316
    32		  2.242975083203717
    33		  2.1308263290435256
    34		  2.024285012591349
    35		  1.9230707619617888
    36		  1

(1, 0, 1, 0, 1, 1, 1, 2, 1, 0, 2, 1, 2, 2, 0, 0, 0, 0, 0)

#Display the results for strong negative rewards, with no reward for the goal state

In [ ]:
#print(V.policy)
#print(V.V)
print("   Policy  V-function")
for i in range(len(V.policy)):
  print(str(i) + ":  " + str(V.policy[i])+"          "+str(V.V[i]))

#Conclusions:
#if we focus on the path starting from S0 (initial state), the path followed to reach the goal is the folowing one:
#S0 -> 4 -> 8 -> 11 -> 13 -> 12 -> 10 -> 7 -> G
#This is the safest path, it avoids all the dead ends, but it is also the longest.

   Policy  V-function
0:  1          -6.731591374218749
1:  2          -19.99899611072055
2:  2          -19.99899611072055
3:  2          -19.99899611072055
4:  1          -6.033254078124999
5:  0          -19.99899611072055
6:  1          -2.8525
7:  2          -1.0
8:  1          -5.298162187499999
9:  0          -96.92123055360274
10:  2          -1.95
11:  1          -4.524381249999999
12:  2          -2.8525
13:  2          -3.709875
14:  0          -199.9899611072055
15:  0          -199.9899611072055
16:  0          -199.9899611072055
17:  0          -199.9899611072055
18:  0          0.0


#Display the results for weak negative rewards, with no reward for the goal state

In [ ]:
#print(V.policy)
#print(V.V)
print("   Policy  V-function")
for i in range(len(V.policy)):
  print(str(i) + ":  " + str(V.policy[i])+"          "+str(V.V[i]))

#Conclusions:
#The path obtained starting from the initial state is similar to the previous path found.
#The safest past is once again prefered.
#We can notice this time that another shortest path is suggested, but still not prefered to the safest and longest path:
# S0 -> 1 -> 2 -> 6  -> 10 -> 7  -> G (shortest path suggested)
# S0 -> 4 -> 8 -> 11 -> 13 -> 12 -> 10 -> 7 -> G (prefered option)

   Policy  V-function
0:  1          -6.731591374218749
1:  0          -16.12352376167754
2:  1          -11.849889799451693
3:  0          -10.494952299451695
4:  1          -6.033254078124999
5:  1          -11.849889799451693
6:  1          -2.8525
7:  2          -1.0
8:  1          -5.298162187499999
9:  0          -11.421202299451695
10:  2          -1.95
11:  1          -4.524381249999999
12:  2          -2.8525
13:  2          -3.709875
14:  0          -19.98990459890339
15:  0          -19.98990459890339
16:  0          -19.98990459890339
17:  0          -19.98990459890339
18:  0          0.0


#Display results for strong positive reward for the goal state and strong negative rewards for dead ends

In [ ]:
#print(V.policy)
#print(V.V)
print("   Policy  V-function")
for i in range(len(V.policy)):
  print(str(i) + ":  " + str(V.policy[i])+"          "+str(V.V[i]))

#Conclusions:
#The path obtained starting from the initial state is similar to the previous path found.
#The safest past is once again prefered.

   Policy  V-function
0:  1          125.94734149745267
1:  2          -19.9994846613859
2:  1          -15.904312499999985
3:  0          -1.0
4:  1          133.62905175448395
5:  1          -11.629312499999994
6:  1          168.61734661385893
7:  2          188.99484661385893
8:  1          141.71506255135898
9:  0          -11.188749999999994
10:  2          178.5448466138589
11:  1          150.22665286385896
12:  2          168.61734661385893
13:  2          159.18622161385895
14:  0          -199.99484661385893
15:  0          -199.99484661385893
16:  0          -199.99484661385893
17:  0          -199.99484661385893
18:  0          199.99484661385893


#Display results for strong positive reward for the goal state, with no strong negative rewards for dead ends

In [ ]:
#print(V.policy)
#print(V.V)
print("   Policy  V-function")
for i in range(len(V.policy)):
  print(str(i) + ":  " + str(V.policy[i])+"          "+str(V.V[i]))

#Conclusions:
#The path obtained starting from the initial state is similar to the previous path found.
#The safest past is once again prefered.
#We can notice this time that another shortest path is suggested, but still not prefered to the safest and longest path:
# S0 -> 1 -> 2 -> 6  -> 10 -> 7  -> G (shortest path suggested)
# S0 -> 4 -> 8 -> 11 -> 13 -> 12 -> 10 -> 7 -> G (prefered option)
#This shortest path meets two potentially dead end states, and is two step shortest than the safest path.
#We can also calculate the probability to reach the goal state for both paths:
# - the safest path meets avoids all the dead ends, so the probability to reach the goal is 1*1*1*1*1*1*1*1 = 1
# - the shortest path suggested visits two states that can lead to a dead end with a probability of 0.5, so the probability to reach the goal is 0.5*0.5*1*1*1*1 = 0.25

   Policy  V-function
0:  1          125.94295793543895
1:  0          22.5562825965729
2:  1          69.59139587333034
3:  0          84.49570837333034
4:  1          133.6246681924702
5:  1          69.59139587333034
6:  1          168.6129630518452
7:  2          188.9904630518452
8:  1          141.7106789893452
9:  0          74.30695837333035
10:  2          178.5404630518452
11:  1          150.2222693018452
12:  2          168.6129630518452
13:  2          159.1818380518452
14:  0          -19.99904630518452
15:  0          -19.99904630518452
16:  0          -19.99904630518452
17:  0          -19.99904630518452
18:  0          199.9904630518452
